In [2]:
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import norm
from scipy.stats import lognorm
import math
from datetime import date
from datetime import datetime
import seaborn as sns

In [ ]:
def histogram(a,values):
    xmin=np.min(a)
    xmax=np.max(a)
    L=100 #the number of segments
    Del=(xmax-xmin)/L
    bin_cents=[xmin+Del/2+k*Del for k in range(L)]
    bin_mins=[xmin+k*Del for k in range(L)]
    bin_maxs=[xmin+(k+1)*Del for k in range(L)]
    counts=np.zeros(L)
    for c1 in range(values):
        k=int(np.floor((a[c1]-xmin)/Del))
        if k>=L:
            counts[L-1]+=1
        else:
            counts[k]+=1
    sns.set_style('whitegrid')
    axes=sns.barplot(x=bin_cents,y=counts/values,palette='bright')
    axes.set(xlabel='Value',ylabel='Frequency')

In [3]:
class Options:
    def __init__(self,symbol, r,T,M,N, start_date, end_date = date.today()):
        self.symbol = symbol
        self.start_date = start_date
        self.end_date = end_date
        self.hist = yf.download(symbol, start=start_date, end=end_date) #Download history for the given symbol
        self.contractData = yf.Ticker(symbol)
        self.returns = np.log(self.hist["Adj Close"] / self.hist["Adj Close"].shift(1)).dropna()
        self.mu = self.returns.mean()
        self.sigma = self.returns.std() 
        self.DF_calls, self.DF_puts = self.contractData.option_chain(self.contractData.options[1])
        self.S0 = self.hist["Adj Close"][-1] # current stock price
        self.r = r # risk-free rate
        self.T = T # time to maturity
        self.M = M # number of time steps
        self.N = N # number of simulations
        #self.q =6T0.03 # annual dividend rate
        self.priceMatrix = np.zeros((self.N, self.M+1))
        
        
    def monteCarlo(self):
        dt = self.T / self.M
        self.priceMatrix[:, 0] = self.S0
        for i in range(self.N):
            for j in range(1, self.M+1):
                self.priceMatrix[i, j] = self.priceMatrix[i, j-1] * np.exp((self.r - 0.5 * self.sigma**2) * dt + self.sigma * np.sqrt(dt) * np.random.normal())
                
    def meanST(self):           
        return self.S0*math.exp((self.r-(self.sigma**2)*0.5)*self.T) * math.exp(0 + self.sigma**2*self.T*0.5)
        
    def plotMonteCarlo(self):
        fig, ax = plt.subplots()
        ax.plot(self.priceMatrix.T)
        ax.set_xlabel("Time")
        ax.set_ylabel("Stock Price")
        ax.set_title("Simulated Stock Price")
        plt.show() 
        
    def plotHistStock(self):
        fig, ax = plt.subplots()
        ax.plot(self.hist["Adj Close"])
        ax.set_xlabel("Date")
        ax.set_ylabel("Stock Price")
        plt.show()
        
    def plotSimulationStock(self):
        histogram((self.priceMatrix.T)[self.M-1,:],self.N)
        
    def plotIdeal(self):
        print(self.S0)
        mu = self.S0*math.exp((self.mu)*self.T)
        print(mu)
        #print(np.mean(self.priceMatrix[:, -1]))
        sigma2 = self.S0**2 * math.exp(self.r*self.T)**2 * (math.exp(self.sigma**2 * self.T)-1)
        print(sigma2)
        #print((self.priceMatrix.T)[self.M-1,:].std()**2)
        deviations = self.priceMatrix[:, -1] - np.mean(self.priceMatrix[:, -1])
        squared_deviations = deviations**2
        mean_squared_deviation = np.mean(squared_deviations)
        print(mean_squared_deviation)
        print(np.std(self.priceMatrix[:,-1]))
        s = np.sqrt(sigma2)
        scale = np.exp(mu)
        x = np.linspace(0, 10, 100)
        pdf = lognorm.pdf(x, s=s, scale=scale)
        plt.plot(x, pdf)
        plt.title('Lognormal Distribution with mu = {} and sigma^2 = {}'.format(mu, s))
        plt.xlabel('X')
        plt.ylabel('PDF')
        plt.show()
        
    def blackScholesCall(self,K):
        # d1 = (np.log(self.S0/K) + (self.r - self.q + self.sigma**2/2)*T) / self.sigma*np.sqrt(T)
        # d2 = d1 - self.sigma* np.sqrt(T)
        # call = self.S0 * np.exp(-self.q*T)* norm.cdf(d1) - K * np.exp(-self.r*T)*norm.cdf(d2)
        d1 = (np.log(self.S0/K) + (self.r  + self.sigma**2/2)*self.T) / self.sigma*np.sqrt(self.T)
        d2 = d1 - self.sigma* np.sqrt(self.T)
        call = self.S0 * norm.cdf(d1) - K * np.exp(-self.r*self.T)*norm.cdf(d2)
        return call
    
    def simulationCall(self,K):
        payoffs = np.maximum(self.priceMatrix[-1]-K, 0)
        # print(self.r)
        # print(T)
        # print(self.priceMatrix[-1]-K)
        option_price = np.mean(payoffs)*np.exp(-self.r*self.T) #discounting back to present value
        return option_price
    
    def blackScholesPut(self,K):                          #Black Scholes Put
        d1 = (np.log(self.S0/K) + (self.r + self.sigma**2/2)*self.T) / self.sigma*np.sqrt(self.T)
        d2 = d1 - self.sigma* np.sqrt(self.T)
        put =  K * np.exp(-self.r*self.T)*norm.cdf(-d2) - self.S0 * norm.cdf(-d1)
        return put
    
    def simulationPut(self,K):
        payoffs = np.maximum(K-self.priceMatrix[-1],0)
        options_price=np.mean(payoffs)*np.exp(-self.r*self.T)
        return options_price
    
    def contractList(self,choice,rng):  #to get multiple contract list table with a given range(rng). Use 'P' for put and 'C' for call in choice parameter.
        superDF_calls,superDF_puts=self.contractData.option_chain(self.contractData.options[0])
        for i in range(1,rng):
                calls1,puts1=self.contractData.option_chain(self.contractData.options[i])
                if choice == 'C':
                    superDF_calls=pd.concat([superDF_calls,calls1],ignore_index=True)
                else:
                    superDF_puts=pd.concat([superDF_puts,puts1],ignore_index=True)
        if choice == 'C':
            return superDF_calls
        else:
            return superDF_puts
        


In [4]:
TSLA = Options("AAPL",0.05,9/365,500,1000,"2019-04-08")

[*********************100%***********************]  1 of 1 completed


In [5]:
print(TSLA.contractList('C',5))
#print(type(TSLA.DF_calls))

          contractSymbol             lastTradeDate  strike  lastPrice     bid  \
0    AAPL230505C00050000 2023-05-02 15:31:21+00:00    50.0     119.95  119.80   
1    AAPL230505C00060000 2023-05-02 17:31:30+00:00    60.0     108.90  109.80   
2    AAPL230505C00065000 2023-05-01 13:53:16+00:00    65.0     105.25  104.80   
3    AAPL230505C00085000 2023-04-04 14:40:18+00:00    85.0      81.35   83.95   
4    AAPL230505C00090000 2023-04-26 13:32:49+00:00    90.0      73.52   79.70   
..                   ...                       ...     ...        ...     ...   
211  AAPL230602C00215000 2023-04-28 18:18:34+00:00   215.0       0.03    0.00   
212  AAPL230602C00225000 2023-05-03 13:30:00+00:00   225.0       0.05    0.00   
213  AAPL230602C00230000 2023-04-18 17:35:11+00:00   230.0       0.03    0.00   
214  AAPL230602C00245000 2023-05-02 18:57:10+00:00   245.0       0.02    0.00   
215  AAPL230602C00250000 2023-04-27 13:30:01+00:00   250.0       0.01    0.00   

        ask    change  perc

In [ ]:
TSLA.monteCarlo()
TSLA.plotMonteCarlo()

In [ ]:
# TSLA.plotHistStock()

In [ ]:
black1 = TSLA.blackScholesCall(50)
print(black1)

In [ ]:
path_len = [1,2,3,5,10,50,100,500,1000]
l = len(path_len)
N_list = np.zeros(l)
black = [black1]*l
for i in range(l):
    TSLA = Options("TSLA",0.05,21/365,500,path_len[i], "2019-04-08")
    for j in range(5):
        TSLA.monteCarlo()
        N_list[i] += TSLA.simulationCall(50)/5
        
# print(N_list)

In [ ]:
plt.plot(path_len, black, color='r', label='black scholes')
plt.plot(path_len, N_list, color='y', label='simulation')

In [ ]:
path_steps = [1,2,3,5,10,20,50,75,100,250,500]
l = len(path_steps)
M_list = np.zeros(l)
black = [TSLA.blackScholesCall(50)]*l
for i in range(l):
    TSLA = Options("TSLA",0.05,9/365,path_steps[i],1000, "2019-04-08")
    for j in range(10):
        TSLA.monteCarlo()
        M_list[i] += TSLA.simulationCall(50)/10
        
print(M_list)

In [ ]:
# path_steps = np.log(path_steps)
plt.plot(path_steps, black, color='r', label='mean')
plt.plot(path_steps, M_list, color='y', label='simulation')

In [ ]:
rates = np.linspace(0.02,0.08,7)
l = len(rates)
r_list = np.zeros(l)
for i in range(len(rates)):
    TSLA = Options("TSLA",rates[i],21/365,500,1000, "2019-04-08")
    for j in range(5):
        TSLA.monteCarlo()
        r_list[i] += TSLA.simulationCall(50)/5

In [ ]:
r_list

In [ ]:
TSLA.meanST()

In [ ]:
#testing data
print(TSLA.blackScholesPut(360))
print(TSLA.simulationPut(360))

In [ ]:
strikePrice=TSLA.DF_puts.strike
l=len(strikePrice)
blkschPut=np.zeros(l)
simPut=np.zeros(l)
diff = []
diff2 = []

for i in range(l):
    blkschPut[i]=TSLA.blackScholesPut(strikePrice[i])
    simPut[i]=TSLA.simulationPut(strikePrice[i])
    diff.append(TSLA.DF_puts['lastPrice'].iloc[i]-blkschPut[i])
    diff2.append(TSLA.DF_puts['lastPrice'].iloc[i]-simPut[i])


 # print(blkschPut,"\n\n\n")
# print(simPut,"\n\n\n")   


plt.plot(strikePrice,blkschPut,label="Black Scholes")
plt.plot(strikePrice,simPut,label="Simulated")

plt.legend()


In [ ]:
count = 0
for i in range(len(diff2)):
    if diff[i]>7 or diff[i]<-7:
        count += 1

Confidence = ((l-count)/l)*100
print(l)
print(Confidence)

In [ ]:
print(TSLA.DF_puts)

In [ ]:
print(TSLA.S0)

In [ ]:
for i in range(l):
    contractSym = TSLA.DF_calls['contractSymbol'].iloc[i]
    date1 = contractSym[len("TSLA"):len("TSLA")+2] + '/' + contractSym[len("TSLA")+2:len("TSLA")+4] + '/' + contractSym[len("TSLA")+4:len("TSLA")+6]
    date2 = datetime.strptime(date1, '%y/%m/%d')
    date3 = date(int('20'+ contractSym[len("TSLA"):len("TSLA")+2]), int(contractSym[len("TSLA")+2:len("TSLA")+4]), int(contractSym[len("TSLA")+4:len("TSLA")+6]))
    print((date3-date.today()).days)